In [1]:
"""
required libraries imported
"""

! pip install category_encoders --quiet
!pip install catboost

import category_encoders as ce

import os, io, sys, random, time, pprint, math, csv
import numpy as np
from catboost import CatBoostClassifier
from numpy import save, load
from itertools import chain
from pylab import rcParams
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import one_hot
from tensorflow.keras.callbacks import LambdaCallback, Callback, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, Sequential
from tensorflow.keras import backend, regularizers
from tensorflow.keras.layers import Activation, Reshape, Dense, Embedding, Dropout, Input, BatchNormalization, concatenate, Flatten, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adadelta, Adagrad, Adamax
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, RepeatedStratifiedKFold,cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 13.3 MB/s eta 0:00:0000:0100:01


In [26]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='##',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'illegaldumps-donotdelete-pr-k6sugbojnndyom'


final = pd.read_csv(cos_client.get_object(Bucket=bucket,Key='Complete_dumpSites.csv')['Body'])
print(f"final has shape {final.shape}")

final_random = pd.read_csv(cos_client.get_object(Bucket=bucket,Key='Complete_randomPoints_1km.csv')['Body'])
print(f"final_random has shape {final_random.shape}")

# testing data
final_test = pd.read_csv(cos_client.get_object(Bucket=bucket,Key='Complete_dumpSites_Test.csv')['Body'])
print(f"final_test has shape {final_test.shape}")

final_random_test = pd.read_csv(cos_client.get_object(Bucket=bucket,Key='Complete_randomPoints_1km_Test.csv')['Body'])
print(f"final_random_test has shape {final_random_test.shape}")

final has shape (44881, 21)
final_random has shape (44545, 21)
final_test has shape (11217, 21)
final_random_test has shape (11133, 21)


In [27]:
#@markdown ## Removing NaN values
"""
removing null values from Continent variables
"""

final = final[final['Continent'].notna()]
final_random = final_random[final_random['Continent'].notna()]

final_test = final_test[final_test['Continent'].notna()]
final_random_test = final_random_test[final_random_test['Continent'].notna()]

In [28]:
#@markdown ## Adding target labels
"""
adding target labels
"""

if 'target' not in final.columns + final_random.columns:
    final['target'] = 1
    final_random['target'] = 0

if 'target' not in final_test.columns + final_random_test.columns:
    final_test['target'] = 1
    final_random_test['target'] = 0

In [29]:
data = pd.concat([final, final_random], axis=0)

data_test = pd.concat([final_test, final_random_test], axis=0)
data.drop(['Lat', 'Lon', 'TrashOutID','Population Density','Population gradient'], axis=1, inplace=True)
data_test.drop(['Lat', 'Lon', 'TrashOutID','Population Density','Population gradient'], axis=1, inplace=True)

data.dropna(inplace=True)
data_test.dropna(inplace=True)

X = data.drop(['target'], axis=1).to_numpy()
Y = data['target'].to_numpy()

test_x = data_test.drop(['target'], axis=1).to_numpy()
test_y = data_test['target'].to_numpy()

data.head()

,Continent,Distance to Road,Distance to Minor Road,Distance to Nearest Venue,Number of Venues,Avg Dist to Venues,1stMostFreq,2ndMostFreq,3rdMostFreq,4thMostFreq,5thMostFreq,1stClosest,2ndClosest,3rdClosest,4thClosest,5thClosest,target
0,Africa,152.071468,47.383710,341,5,654.2,Bar,Tennis Court,Restaurant,Student Center,Bus Station,Restaurant,Bar,Student Center,Tennis Court,Bus Station,1
1,Africa,188.271287,10.439037,1500,0,3000.0,None,None,None,None,None,None,None,None,None,None,1
2,Africa,473.134755,0.000000,496,4,620.0,Auto Workshop,Beer Garden,Restaurant,Gas Station,None,Restaurant,Gas Station,Auto Workshop,Beer Garden,None,1
3,Africa,2500.000000,0.000000,61,1,61.0,Shopping Mall,None,None,None,None,Shopping Mall,None,None,None,None,1
4,Africa,6.404573,16.189577,1500,0,3000.0,None,None,None,None,None,None,None,None,None,None,1


In [30]:
from numpy import mean
from numpy import std

model = CatBoostClassifier(verbose=0, n_estimators=100,cat_features=[0,6,7,8,9,10,11,12,13,14,15])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.721 (0.005)


In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={}

In [65]:
import requests
from collections import Counter
def get_venue_features(lat,long):
    

    url =f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius=1000&sort=DISTANCE&limit=50"

    headers = {
        "accept": "application/json",
        "Authorization": "fsq3fBY9YJDgjYrpbw5+BAhEYcoS9NFk6z7ZDR9yWn5vGPU="
    }

    response = requests.get(url, headers=headers)
    data=response.json()
    venues={}
    for items in data['results']:
#         print(items['distance'])
#         print(items['categories'][0]['name'])
        if items['distance']<=1000:
            venues[items['categories'][0]['name']]=items['distance']
#     print(venues)
    min_distance = float('inf')

# Iterate through the dictionary to find the minimum distance
    for venue, distance in venues.items():
        if distance < min_distance:
            min_distance = distance
            nearest_venue = venue
    total_distance = 0
    venue_count = 0

    # Iterate through the dictionary to calculate the total distance and count of venues
    for distance in venues.values():
        total_distance += distance
        venue_count += 1

    # Calculate the average distance
    average_distance = total_distance / venue_count
    # Print the nearest venue and its distance
#     print(f"Nearest Venue: {nearest_venue}")
#     print(f"Distance to Nearest Venue: {min_distance} km")
    venue_types=[venue for venue in venues.keys()]
    venue_type_counts=Counter(venue_types)
    most_common_venues=[venue for venue,_ in venue_type_counts.most_common(5)]
#     print(most_common_venues)
    nearest_venues=sorted(venues,key=venues.get)
#     print(venues[0])
    return nearest_venues[:5],most_common_venues,min_distance,average_distance

In [66]:
print(get_venue_features(12.9003214,77.6526815))

(['Bakery', 'Health and Medicine', 'Pizzeria', 'Miscellaneous Store', 'Restaurant'], ['Bakery', 'Health and Medicine', 'Sports and Recreation', 'Pizzeria', 'Retail'], 327, 695.3)


In [18]:
!pip install geopy
!pip install folium
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 59.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 39.0 MB/s eta 0:00:0000:0100:01


In [19]:
import osmnx as ox
import folium
from shapely.geometry import Point
from geopy.distance import geodesic

def get_nearest_road(target_latitude,target_longitude):
    # Replace these coordinates with the latitude and longitude of your target location
#     target_latitude = 19.194021514473864  # Example latitude
#     target_longitude = 73.17782990963588   # Example longitude

    # Define a buffer distance (in meters) for the nearest road search
    buffer_distance = 1000  # Adjust as needed

    # Download OSM data for the target area within the specified buffer distance
    graph = ox.graph_from_point(
        (target_latitude, target_longitude),
        network_type='all',  # Include all types of roads
        dist=buffer_distance,  # Buffer distance in meters
        simplify=True  # Simplify the network for analysis
    )

    # Create a Shapely Point object representing your target location
    target_point = Point(target_longitude, target_latitude)

    # Find the nearest edge (road segment) to your target location
    nearest_edge = ox.distance.nearest_edges(graph, X=target_longitude, Y=target_latitude)

    # Extract the coordinates of the nearest edge's start and end nodes
    start_node_coords = graph.nodes[nearest_edge[0]]['y'], graph.nodes[nearest_edge[0]]['x']
    end_node_coords = graph.nodes[nearest_edge[1]]['y'], graph.nodes[nearest_edge[1]]['x']

    # Calculate the distance between your location and the nearest road using geodesic distance in kilometers
    distance_to_nearest_edge = min(
        geodesic((target_latitude, target_longitude), start_node_coords).kilometers,
        geodesic((target_latitude, target_longitude), end_node_coords).kilometers
    )

    # Create a Folium map centered on the target location
    m = folium.Map(location=[target_latitude, target_longitude], zoom_start=15)

    # Add the target location as a marker on the map
    folium.Marker([target_latitude, target_longitude], tooltip='Target Location').add_to(m)

    # Add the nearest edge (road segment) as a GeoJSON layer on the map
    folium.PolyLine(locations=[start_node_coords, end_node_coords],
                     color='red', weight=5, opacity=0.7).add_to(m)

    # Save the map to an HTML file
    m.save('map_red.html')

    # Print the distance from your location to the nearest road in kilometers
    print(f'Distance to Nearest Road (in kilometers): {distance_to_nearest_edge} km')
    return distance_to_nearest_edge

In [21]:
get_nearest_road(12.9003214,77.6526815)

Distance to Nearest Road (in kilometers): 0.018222017978535884 km


0.018222017978535884

In [34]:
!pip install geopy pycountry_convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.4 MB/s eta 0:00:0000:010:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 28.3 MB/s eta 0:00:00
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681847 sha256=6440afd526419540b56f20720daf8b478051de7dc6fee6accc66cad1e50eae41
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry


In [42]:
from geopy.geocoders import Nominatim
import pycountry_convert as pc
from typing import Dict
def get_continent(lat,long):
    

# Initialize the Nominatim geocoder
    geolocator = Nominatim(user_agent="continent_locator")

    # Define the coordinates (latitude and longitude)
    

    # Reverse geocode the coordinates to get location information
    location = geolocator.reverse(f"{lat}, {long}")

    if location:
        address = location.raw.get("address", {})
        country=address['country']
        country_code:str=pc.country_name_to_country_alpha2(country,cn_name_format='default')
        
        continent_code:str = pc.country_alpha2_to_continent_code(country_code)
        # Define a mapping of continent codes to continent names
        continent_mapping = {
            'AF': 'Africa',
            'AN': 'Antarctica',
            'AS': 'Asia',
            'EU': 'Europe',
            'NA': 'North America',
            'OC': 'Oceania',
            'SA': 'South America'
        }

        # Define the continent code you want to look up
        
        # Look up the continent name based on the code
        continent_name = continent_mapping.get(continent_code, 'Unknown')

        

        print(f"The continent at coordinates {lat}, {long} is: {continent_name}")
    else:
        print("Unable to determine the continent for the given coordinates.")


In [43]:
get_continent(12.9003214,77.6526815)

The continent at coordinates 12.9003214, 77.6526815 is: Asia


In [70]:
import pandas as pd
def get_prediction(lat,long):
    continent=get_continent(lat,long)
    nearest_venues,frequent_venues,dist,avg_dist=get_venue_features(lat,long)
    df=pd.DataFrame('Continent':[continent],'Distance to Road':[get_nearest_road(lat,long)],'Distance to Minor Road':[get_nearest_road(lat,long)],'Distance to Nearest Venue':[dist],'Avg Dist to Venues':[avg_dist],'1stMostFreq':[frequent_venues[0]],'2ndMostFreq':[frequent_venues[1]],'3rdMostFreq':[frequent_venues[2]],'4thMostFreq':[frequent_venues[3]],'5thMostFreq':[frequent_venues[4]],'1stClosest':[nearest_venues[0]],'2ndClosest':[nearest_venues[1]],
                    '3rdClosest':[nearest_venues[2]],'4thClosest':[nearest_venues[3]],'5thClosest':[nearest_venues[4]])
    print(model.predict(df))
    

SyntaxError: invalid syntax (538868001.py, line 5)

In [ ]:
get_prediction(12.9003214,77.6526815)